In [2]:
%pylab inline 

Populating the interactive namespace from numpy and matplotlib


In [3]:
cd ../TileEM/

/home/jlee782/crowd-seg/analysis/TileEM


In [4]:
# Run the PR calculation for PixelEM on various thresholds based on  the adjacency and the non-adjacency ML construction
# Creates a Pixel_PR.csv in each of the base_dir/dir_name directory
import pandas as pd
import pickle as pkl
from qualityBaseline import *
from calc_Tstar import *
import os
import time
from PixelEM import *
base_dir= "pixel_em"
start = time.time()
worker_Nbatches={5:10,10:8,15:6,20:4,25:2,30:1}
sampleN_lst=worker_Nbatches.keys()
from TileEM_plot_toolbox import *
start = time.time()
PR_pixelEM = pd.read_csv("pixel_em/full_PRJ_table.csv")
PR_pixelEM_GT = pd.read_csv("pixel_em/GTfull_PRJ_table.csv")
#PR_pixelEM_GTLSA = pd.read_csv("pixel_em/GTLSAfull_PRJ_table.csv")
df = pd.read_csv("../computed_my_COCO_BBvals.csv",index_col=0)
tbl=[]
col_lst=[]
i=0
cols = [u'Num Points',u'Area Ratio',u'Jaccard [Self]', u'Precision [Self]', u'Recall [Self]']
for Nworker in sampleN_lst:
    for batch_num in range(worker_Nbatches[Nworker]):
        dir_name = "{0}workers_rand{1}".format(Nworker,batch_num)
        print "Working on :", dir_name
        #os.chdir(base_dir+"/"+dir_name)
        tmp_tbl=[]
        #for i,fname in enumerate(glob("obj*")):
        #    objid=int(fname[3:])
        object_lst = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 36, 37, 38, 39, 42, 43, 44, 45, 46, 47]
        for objid in tqdm(object_lst):
            tmp_tbl=[objid,Nworker,batch_num]
            if i==0: col_lst = ["object_id","Nworker","batch_num"]
            #Summarization Based metrics
            workers = ast.literal_eval([line for line in open("pixel_em/"+dir_name+"/obj{}/worker_ids.json".format(objid))][0])
            filtered_df = df[(df["worker_id"].isin(workers))&(df["object_id"]==objid)] #only look at summarization scores of sampled workers
            if len(filtered_df)!=0:
                for attr in cols:
                    if i==0:
                        col_lst.extend(["P [{}]".format(attr),"R [{}]".format(attr),"J [{}]".format(attr)])
                    best_worker_BB = filtered_df[filtered_df[attr]==filtered_df[attr].max()]
                    best_worker_id = int(best_worker_BB["worker_id"].sample(1).values)
                    best_worker_mask = pkl.load(open("pixel_em/obj{}/mask{}.pkl".format(objid,best_worker_id)))
                    gt_mask = pkl.load(open("pixel_em/obj{}/gt.pkl".format(objid)))
                    p,r,j = get_precision_recall_jaccard(best_worker_mask, gt_mask)
                    tmp_tbl.extend([p,r,j])
        tbl.append(tmp_tbl)
        i+=1

/home/jlee782/.local/lib/python2.7/site-packages/matplotlib/__init__.py:1401: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)
  0%|          | 0/44 [00:00<?, ?it/s]

Working on : 5workers_rand0


  0%|          | 0/44 [00:00<?, ?it/s]         

Working on : 5workers_rand1


  0%|          | 0/44 [00:00<?, ?it/s]         

Working on : 5workers_rand2


 27%|██▋       | 12/44 [01:16<03:24,  0.16it/s]

KeyboardInterrupt: 

In [76]:
# df = pd.DataFrame(tbl,columns=col_lst)
# df.to_csv("pixel_em/summarization_PRJ_table.csv")
# end = time.time()
# print "Time Elapsed: ", end-start

Summarization score takes the longest to compute 

In [38]:
# Contain PRJs for best summarization score, Basic Pixel EM, Pixel MV 
Pixel_PR =pd.read_csv("Pixel_PR.csv",index_col=0)

In [14]:
# PR_pixelEM = pd.read_csv("pixel_em/full_PRJ_table.csv")
PR_pixelEM_GT = pd.read_csv("pixel_em/GTfull_PRJ_table.csv")


In [21]:
for thresh in [-4,-2,0,2,4]:
    PR_pixelEM_GTi =PR_pixelEM_GT[(PR_pixelEM["thresh"]==thresh)]
#                         if len(PR_pixelEMi)!=0:
#                             PixelEMP= float(PR_pixelEMi["EM_precision"])
#                             PixelEMR= float(PR_pixelEMi["EM_recall"])
#                             PixelEMJ=float(PR_pixelEMi["EM_jaccard"])
#                             tmp_tbl.extend([PixelEMP,PixelEMR,PixelEMJ])

In [70]:
updated_Pixel_PR= Pixel_PR
for thresh in [-4,-2,0,2,4]:
    PR_pixelEM_GTi =PR_pixelEM_GT[PR_pixelEM_GT["thresh"]==thresh]
    PR_pixelEM_GTi = PR_pixelEM_GTi.rename(index=str,columns={'EM_precision':'P [GT Pixel EM thres={}]'.format(thresh),\
                       'EM_recall':'R [GT Pixel EM thres={}]'.format(thresh),\
                       'EM_jaccard':'J [GT Pixel EM thres={}]'.format(thresh),
                       'objid':"object_id",\
                       'num_workers':'Nworker',\
                        'sample_num':'batch_num'})
    PR_pixelEM_GTi = PR_pixelEM_GTi.drop('thresh',axis=1)
    updated_Pixel_PR= updated_Pixel_PR.merge(PR_pixelEM_GTi)

In [71]:
Pixel_PR.head()

,object_id,Nworker,batch_num,P [Num Points],R [Num Points],J [Num Points],P [Area Ratio],R [Area Ratio],J [Area Ratio],P [Jaccard [Self]],...,J [PixelEM thres=4],P [PixelEM thres=10],R [PixelEM thres=10],J [PixelEM thres=10],P [PixelEM thres=-10],R [PixelEM thres=-10],J [PixelEM thres=-10],P [MV],R [MV],J [MV]
0,1,5,0,0.905567,0.990299,0.897605,0.905567,0.990299,0.897605,0.976263,...,0.949210,0.994513,0.920677,0.916024,0.896327,0.997174,0.894056,0.977603,0.980179,0.958651
1,2,5,0,0.929051,0.977324,0.909447,0.929051,0.977324,0.909447,0.945597,...,0.927700,0.980922,0.833909,0.820600,0.901088,0.994594,0.896696,0.949744,0.976273,0.928316
2,3,5,0,0.942595,0.944328,0.892972,0.942595,0.944328,0.892972,0.942595,...,0.862237,0.975768,0.867122,0.848843,0.921957,0.964811,0.891964,0.961358,0.914653,0.882219
3,4,5,0,0.928049,0.961515,0.894811,0.447659,0.933030,0.433723,0.952381,...,0.883256,0.978738,0.878788,0.862325,0.881558,0.987879,0.872124,0.926200,0.946970,0.880530
4,5,5,0,0.935396,0.851074,0.803826,0.864442,0.927636,0.809832,0.977446,...,0.814851,0.986301,0.764633,0.756598,0.831643,0.951593,0.797888,0.959925,0.881452,0.850167


In [72]:
PR_pixelEM_GTi.head()

,Nworker,batch_num,object_id,P [GT Pixel EM thres=4],R [GT Pixel EM thres=4],J [GT Pixel EM thres=4]
1,5,0,1,0.977603,0.980179,0.958651
8,5,0,11,0.869027,0.911510,0.801415
13,5,0,13,0.963661,0.962122,0.928437
18,5,0,14,0.936997,0.984495,0.923371
21,5,0,3,0.942751,0.934086,0.883946


In [75]:
updated_Pixel_PR.mean()

object_id                   22.895203
Nworker                     12.140221
batch_num                    3.084871
P [Num Points]               0.869995
R [Num Points]               0.900022
J [Num Points]               0.810675
P [Area Ratio]               0.755943
R [Area Ratio]               0.918613
J [Area Ratio]               0.732682
P [Jaccard [Self]]           0.952359
R [Jaccard [Self]]           0.954361
J [Jaccard [Self]]           0.911212
P [Precision [Self]]         0.972448
R [Precision [Self]]         0.842063
J [Precision [Self]]         0.820319
P [Recall [Self]]            0.859022
R [Recall [Self]]            0.974735
J [Recall [Self]]            0.838036
P [PixelEM thres=-4]         0.930054
R [PixelEM thres=-4]         0.945525
J [PixelEM thres=-4]         0.881729
P [PixelEM thres=-2]         0.940721
R [PixelEM thres=-2]         0.936541
J [PixelEM thres=-2]         0.883642
P [PixelEM thres=0]          0.944914
R [PixelEM thres=0]          0.931887
J [PixelEM t